<a href="https://colab.research.google.com/github/shivam-daksh/soil-classification-and-plant-recommendation-system/blob/main/Soil_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [ ]:
os.listdir("/content/data/train")
shutil.rmtree("/content/data/train/.ipynb_checkpoints")

In [ ]:
shutil.copytree("/content/drive/MyDrive/AI ML DATA/Soil-Dataset","/content/soil-data")

In [ ]:
# shutil.move("/content/yellow.jpg","/content/predict_data")

'/content/predict_data/yellow.jpg'

In [ ]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20
NUM_CLASSES = 4

In [ ]:
# data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [ ]:
data_dir = "/content/data/train"
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)  # No augmentation for test data

test_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # Since it's a binary classification problem
    shuffle=False  # Don't shuffle the data during testing
)

Found 1129 images belonging to 4 classes.
Found 281 images belonging to 4 classes.
Found 1410 images belonging to 4 classes.


In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

Epoch 1/20
35/35 [==============================] - 23s 618ms/step - loss: 1.5101 - accuracy: 0.5679 - val_loss: 1.1512 - val_accuracy: 0.5469
Epoch 2/20
35/35 [==============================] - 21s 611ms/step - loss: 0.5881 - accuracy: 0.7293 - val_loss: 1.3838 - val_accuracy: 0.6016
Epoch 3/20
35/35 [==============================] - 22s 618ms/step - loss: 0.6009 - accuracy: 0.7356 - val_loss: 1.1391 - val_accuracy: 0.5977
Epoch 4/20
35/35 [==============================] - 22s 634ms/step - loss: 0.4751 - accuracy: 0.8095 - val_loss: 1.0034 - val_accuracy: 0.6328
Epoch 5/20
35/35 [==============================] - 21s 611ms/step - loss: 0.3914 - accuracy: 0.8514 - val_loss: 1.1129 - val_accuracy: 0.6641
Epoch 6/20
35/35 [==============================] - 22s 622ms/step - loss: 0.3767 - accuracy: 0.8614 - val_loss: 0.8567 - val_accuracy: 0.6758
Epoch 7/20
35/35 [==============================] - 21s 612ms/step - loss: 0.3358 - accuracy: 0.8687 - val_loss: 1.0658 - val_accuracy: 0.6953

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
print("Test Accuracy:", test_accuracy)

44/44 [==============================] - 5s 119ms/step - loss: 0.3499 - accuracy: 0.8949
Test Accuracy: 0.8948863744735718


In [ ]:
    img_path = "/content/red.jpeg"
    soils = ["Black", "Clay", "Red","Alluvial"]
    # Load the image
    img = image.load_img(img_path, target_size=(224, 224))

    # Convert image to numpy array
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize

    # Make prediction
    prediction = model.predict(img_array)
    soil_index = np.argmax(prediction)
    soils[soil_index]

1/1 [==============================] - 0s 44ms/step


'Red'

In [ ]:
from tensorflow.keras.preprocessing import image

# Defining Soils
soils = ["Alluvial","Black", "Clay", "Red"]

# Assuming model is already defined and loaded

# Directory containing images
img_dir = "/content/drive/MyDrive/AI ML DATA/Soil-Dataset/soil-pred"

# Load and predict for each image
for file in os.listdir(img_dir):
    img_path = os.path.join(img_dir, file)

    # Load the image
    img = image.load_img(img_path, target_size=(224, 224))

    # Convert image to numpy array
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize

    # Make prediction
    prediction = model.predict(img_array)
    soil_index = np.argmax(prediction)
    print("Image:", file, "- Predicted Soil:", soils[soil_index])


1/1 [==============================] - 0s 22ms/step
Image: peat.jpg - Predicted Soil: Black
1/1 [==============================] - 0s 17ms/step
Image: laterite.jpg - Predicted Soil: Red
1/1 [==============================] - 0s 18ms/step
Image: black-soil.jpg - Predicted Soil: Black
1/1 [==============================] - 0s 17ms/step
Image: yellow.jpg - Predicted Soil: Alluvial
1/1 [==============================] - 0s 18ms/step
Image: cinder.jpg - Predicted Soil: Black


In [ ]:
# Save the model
model.save('/content/drive/MyDrive/AI ML DATA/Models/soil_classification_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#  **Plant Recommendation**

In [ ]:
# Sample plant data (replace with actual data)
plant_data = {
    'Tomato': {'soil_type': 'loamy', 'sunlight': 'full sun', 'moisture': 'moderate', 'pH': '6.0-7.0'},
    'Carrot': {'soil_type': 'sandy', 'sunlight': 'full sun', 'moisture': 'well-drained', 'pH': '6.0-6.8'},
    'Spinach': {'soil_type': 'sandy loam', 'sunlight': 'partial shade', 'moisture': 'moist', 'pH': '6.5-7.5'},
}

def recommend_plants(soil_type):
    recommended_plants = []
    for plant, data in plant_data.items():
        if data['soil_type'] == soil_type:
            recommended_plants.append(plant)
    return recommended_plants

classified_soil_type = 'loamy'



#

# **Soil Classification Model Version 2.0**

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import layers, models

In [ ]:
import zipfile
import shutil

In [ ]:
# Constants
IMAGE_SIZE = (224, 224)  # Example size, adjust according to your images
BATCH_SIZE = 32
NUM_CLASSES = 5
NUM_EPOCHS = 10
LEARNING_RATE = 0.001

In [ ]:
with zipfile.ZipFile('/content/drive/MyDrive/AI ML DATA/Soil-Dataset/New/data.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [ ]:
# shutil.rm
os.remove("/content/data/valid/_classes.csv")

In [ ]:
# Load dataset
def load_dataset(data_dir):
    image_paths = []
    labels = []

    for folder_name in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder_name)
        if os.path.isdir(folder_path):
            for image_name in os.listdir(folder_path):
                if image_name.endswith('.jpg'):
                    image_path = os.path.join(folder_path, image_name)
                    image_paths.append(image_path)
                    labels.append(pd.read_csv(os.path.join(folder_path, f'_classes.csv')))
    return image_paths, labels

In [ ]:
import os
import shutil
import pandas as pd
def move_images_to_class_folders(data_dir, subset):
    subset_folder = os.path.join(data_dir, subset)
    for image_name in os.listdir(subset_folder):
        if image_name.endswith('.jpg'):
            image_path = os.path.join(subset_folder, image_name)
            class_file_path = os.path.join(subset_folder, '_classes.csv')

            if os.path.isfile(class_file_path):
                classes_df = pd.read_csv(class_file_path, index_col='filename')
                image_class = classes_df.loc[image_name, classes_df.columns[1:]].idxmax()

                class_folder = os.path.join(subset_folder, image_class)
                os.makedirs(class_folder, exist_ok=True)

                shutil.move(image_path, os.path.join(class_folder, image_name))
                print(f"Moved {image_name} to {image_class} folder.")
            else:
                print(f"No class file found for {image_name} in {subset}.")

def main():
    data_dir = 'data'  # Adjust the path accordingly

    subsets = ['train', 'test', 'valid']
    for subset in subsets:
        move_images_to_class_folders(data_dir, subset)

if __name__ == "__main__":
    main()

Moved clay-soil-cut-red-color-clay-clay-soil-cut-red-color-clay-184254602_jpg.rf.4b0e83cdcb2e543f1810cf01d470e4a2.jpg to  Clay folder.
Moved Clay_40_jpg.rf.c0ff1567ce1a16572a940cd840efb69e.jpg to  Clay folder.
Moved Copy-of-Sample12-0_jpg.rf.9fff13441cbaf3a7727cf52e79c165c8.jpg to  Clay folder.
Moved Clay_45_jpg.rf.c60d59ef97f0cd2eaf1460156e6b8dad.jpg to  Clay folder.
Moved Copy-of-images281-1-_jpg.rf.4d4b215288eae9d3d18698c35e0c67e0.jpg to  Black folder.
Moved Black_7_jpg.rf.65ba6f78347e1a5229f19f7fbbbf7abe.jpg to  Black folder.
Moved Copy-of-red3_jpg.rf.85a311a6ae3d5d50e6c5d9a1cd289382.jpg to  Red folder.
Moved Copy-of-IMG-20220531-WA0030_jpg.rf.cf9d9a6738f448065ac21c14300f1949.jpg to  soil folder.
Moved Clay_16_jpg.rf.22ed757e4f68d1fecbbab7585b7afdf1.jpg to  Clay folder.
Moved Alluvial_41_jpg.rf.514452a0077a2cf9810179112c55beac.jpg to  soil folder.
Moved Copy-of-images413-1-_jpg.rf.dccb6c72316f1a8eac54a94ba4ebeac8.jpg to  Black folder.
Moved Red_18_jpg.rf.1ebe62ab45fbeaecec7ff1c317e

In [ ]:
# Load dataset
image_paths, labels = load_dataset('data/train')  # Adjust the path accordingly

# Create folders for each class
classes = labels[0].columns[:]  # Extract class names from the first label
for class_name in classes:
    os.makedirs(os.path.join('class_folders', class_name), exist_ok=True)

# Move images to respective class folders
for image_path, label in zip(image_paths, labels):
    image_name = os.path.basename(image_path)
    class_name = label.columns[label.iloc[0].values == 1][0]  # Get class name where value is 1
    destination_folder = os.path.join('data1', class_name)
    shutil.copy(image_path, os.path.join(destination_folder, image_name))

IndexError: list index out of range

In [ ]:
# Preprocess image
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, IMAGE_SIZE)
    image = image.astype('float32') / 255.0  # Normalize pixel values
    return image


In [ ]:
# Load and preprocess dataset
image_paths, labels = load_dataset('data/train')  # Adjust the path accordingly
images = np.array([preprocess_image(image_path) for image_path in image_paths])
labels = np.array(labels)


In [ ]:
# Split dataset into train and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
# Define CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

In [ ]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Train model
model.fit(train_images, train_labels, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_data=(val_images, val_labels))